In [ ]:
import odr_env
import json

# Find all raman files in the provided datarecord list
raman_files = {}
for dr_id in _odr_datarecord_list:
    # If this datarecord has files that have been uploaded to it...
    file_list = odr_env.getFileList(dr_id)
    if ( len(file_list) > 0 ):
        dr_data = odr_env.getDatarecordData(dr_id)
        dr_name = dr_data['datarecords'][0]['datarecord_name']
        rruff_id = dr_data['datarecords'][0]['external_id']

        datarecord_key = dr_name + ' ' + rruff_id

        # Store file ids by name of parent datarecord
        raman_files[datarecord_key] = {}

        # Append all file ids to this list
        for file_id, file_data in file_list.items():
            # Organize by wavelength
            filename = file_data['filename'].split('__')
            file_type = filename[3]
            raman_files[datarecord_key][file_type] = int(file_id)

#print( json.dumps(raman_files, indent=4) )

In [ ]:
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

for sample_name, files in raman_files.items():
    # Create a new plot for this sample
    fig = plt.figure(figsize=(10, 5), dpi=100)
    lines = []
    axis_num = 0
    first_axis = ''
    
    # For each file the sample has...
    current_axis = ''
    for file_type in sorted(files):    # ensure '532' comes first
        
        if ( axis_num == 0 ):
            # Do certain things if this is the first axis for the graph
            first_axis = fig.add_subplot(111)
            first_axis.set_ylabel('Intensity')
            first_axis.set_xlabel('Raman Shift (cm-1)')
            first_axis.get_yaxis().set_ticks([])    # clear tick marks
        else:
            # The rest of the graph axis should work with this?
            current_axis = first_axis.twinx()
            current_axis.yaxis.set_visible(False)    # don't display axis at all

        # ...download the data for each file from ODR
        file_data = odr_env.downloadFile( files[file_type] )
        graph_data = np.genfromtxt(BytesIO(file_data), delimiter=",")    # convert bytes into array
        
        # Split the 2D array into 1D arrays for graphing
        x1 = graph_data[:,0]    # for each sub-array, get the first element of the sub-array and make a new array out of them
        y1 = graph_data[:,1]    # same as above, but with the second element instead
        
        # Need different colors, otherwise it'll just be a mess
        line_color = 'C' + str(axis_num)
        axis_num += 1

        # Add the data to the plot
        if ( axis_num == 1 ):    # first axis that's been drawn, second will have axis_num == 2 by now
            line = first_axis.plot(x1, y1, color=line_color, label='unoriented (' + file_type + 'nm)')
            lines.append( line[0] )    # line itself is a list, don't want lines to be a list of lists
        else:
            line = current_axis.plot(x1, y1, color=line_color, label='unoriented (' + file_type + 'nm)')
            lines.append( line[0] )

    # Add each line to the plot
    labels = [line.get_label() for line in lines]
    current_axis.legend(lines, labels, loc='upper left')    # only have the last axis apply a legend...will be overwritten otherwise
    
    plt.title(sample_name)
    plt.show()